# Fundamentals of Artificial Intelligence Programme (2024/25 Q1)
## Multiobjective optimization for decision support
Dr. Jazmin Zatarain Salazar

In this assignment, we use two python libraries, one for exploratory modeling analysis [The EMA workbench](https://emaworkbench.readthedocs.io/en/latest/) and one for multiobjective optimization [Project Platypus](https://platypus.readthedocs.io/en/latest/), you can install both with pip as follows: 

In [2]:
!pip install ema_workbench platypus-opt

# The Lake model

The goal of this assignment is to demonstrate the use of multiobjective evolutionary optimization, to learn how to visualize and interpret the optimization results, and to use performance metrics to assess the results obtained via multiobjective evolutionary optimization.  We will use the lake problem as a test case, this is a classic problem initially developed by Carpenter et al. (1999) where the population of a city has to decide the amount of annual pollution it will release into a lake.  In this exercise, we will use the adapted version in [Quinn et al. 2017](https://www-sciencedirect-com.tudelft.idm.oclc.org/science/article/pii/S1364815216302250?casa_token=wBdtfic9L4YAAAAA:Oum4EU3ob7bgKoCK2WhEFfOfIGUrsXpvp7PSvnRyMbDSF8lHXdf48H5hMe6d0DS1zChv5IhO) were the problem is defined as a state-based control problem (where actions are a function of the state of the system).  In this case, the 'action' is the Phosphorous (P) emissions which are optimized to balance the economic benefits and the quality of the lake. Since this is a multi-objective problem,  we need a flexible function to map the states to actions, so we use radial basis functions to parameterize the emission control policies. In fact, the MOEA will search for the optimal radii, centers and weights that yield good performance for the objectives of the lake model described below.  See the paper for more details about the problem formulation.

The model is defined by the following equation:

$
    X_{(t+1)}=X_t+a_t+\frac{(X_t^q)}{(1+X_t^q )}- bX_t+\epsilon_t
$

where $X_t$ is the pollution at time $t$, $a_t$ is the rate of anthropogenic pollution at time $t$, $b$ is the lake’s natural removal rate, $q$ is the lake's natural recycling rate, $\epsilon_t$ is the rate of natural pollution at time $t$. The rate of anthropogenic pollution $a_t$ is the decision variable where $a_t \in [0,0.1]$. 

There are four outcomes of interest. The first is the average concentration of phosphor in the lake. 

$
    f_{phosphorus}=  \frac{1}{\left\vert{T}\right\vert} \sum\limits_{t\in{T}} X_t 
$

where $\left\vert{T}\right\vert$ is the cardinality of the set of points in time. 
The second objective is the economic benefit derived from polluting the lake defined as the discounted benefit of pollution minus the costs of having a polluted lake.

$ f_{economic} = \sum\limits_{t \in {T}}\alpha a_t \delta^t$

where $\alpha$ is the utility derived from polluting and $\delta$ is the discount rate. By default, $\alpha$ is 0.04.
The third objective is related to the year over year change in the anthropogenic pollution rate. 

$ f_{inertia} = \frac{1}{\left\vert{T}\right\vert-1}\sum \limits_{t=1}^{\left\vert{T}\right\vert}I(|a_{t}-a_{t-1} |>\tau)$ 

where $I$ is an indicator function that is 0 if the statement is false, and 1 if the statement is true, $\tau$ is the threshold that is deemed undesirable, and is for illustrative purposes et to 0.2. Effectively, f_{inertia} is the fraction of years where the absolute value of the change in anthropogenic pollution is larger then $\tau$.
The fourth objective is the fraction of years where the pollution in the lake is below the critical threshold.

$
    f_{reliability} =  \frac{1}{\left\vert{T}\right\vert} \sum\limits_{t \in T}I(X_{t}<X_{crit} ) 
$

where $I$ is an indicator function that is 0 if the statement is false, and 1 if the statement is true, $X_{crit}$ is the critical threshold of pollution and is a function of both $b$ and $q$.

The lake problem is characterized by both stochastic uncertainty and deep uncertainty. The stochastic uncertainty arises from the natural inflow. To reduce this stochastic uncertainty, multiple replications are performed and the average over the replication is taken. Deep uncertainty is presented by uncertainty about the mean $\mu$ and standard deviation $sigma$ of the lognormal distribution characterizing the natural inflow, the natural removal rate of the lake $\beta$, the natural recycling rate of the lake $q$, and the discount rate $\delta$. The table below specifies the ranges for the deeply uncertain factors, as well as their best estimate or default values. 

# Lake model implementation in python

In [8]:
import math
import numpy as np
from scipy.optimize import brentq


def get_antropogenic_release(xt, c1, c2, r1, r2, w1):
    '''

    Parameters
    ----------
    xt : float
         polution in lake at time t
    c1 : float
         center rbf 1
    c2 : float
         center rbf 2
    r1 : float
         ratius rbf 1
    r2 : float
         ratius rbf 2
    w1 : float
         weight of rbf 1

    Returns
    -------
    float

    note:: w2 = 1 - w1

    '''

    rule = w1 * (abs(xt - c1) / r1) ** 3 + (1 - w1) * (abs(xt - c2) / r2) ** 3
    at1 = max(rule, 0.01)
    at = min(at1, 0.1)

    return at


def lake_model(b=0.42, q=2.0, mean=0.02,
               stdev=0.001, delta=0.98, alpha=0.4,
               nsamples=100, myears=100, c1=0.25,
               c2=0.25, r1=0.5, r2=0.5,
               w1=0.5, seed=123):
    '''runs the lake model for nsamples stochastic realisation using
    specified random seed.

    Parameters
    ----------
    b : float
        decay rate for P in lake (0.42 = irreversible)
    q : float
        recycling exponent
    mean : float
            mean of natural inflows
    stdev : float
            standard deviation of natural inflows
    delta : float
            future utility discount rate
    alpha : float
            utility from pollution
    nsamples : int, optional
    myears : int, optional
    c1 : float
    c2 : float
    r1 : float
    r2 : float
    w1 : float
    seed : int, optional
           seed for the random number generator

    Returns
    -------
    tuple

    '''
    np.random.seed(seed)
    Pcrit = brentq(lambda x: x ** q / (1 + x ** q) - b * x, 0.01, 1.5)

    X = np.zeros((myears,))
    average_daily_P = np.zeros((myears,))
    reliability = 0.0
    inertia = 0
    utility = 0

    for _ in range(nsamples):
        X[0] = 0.0
        decision = 0.1

        decisions = np.zeros(myears, )
        decisions[0] = decision

        natural_inflows = np.random.lognormal(
            math.log(mean ** 2 / math.sqrt(stdev ** 2 + mean ** 2)),
            math.sqrt(math.log(1.0 + stdev ** 2 / mean ** 2)),
            size=myears)

        for t in range(1, myears):
            # here we use the decision rule
            decision = get_antropogenic_release(X[t - 1], c1, c2, r1, r2, w1)
            decisions[t] = decision

            X[t] = (1 - b) * X[t - 1] + X[t - 1] ** q / (1 + X[t - 1] ** q) + decision + \
                   natural_inflows[t - 1]
            average_daily_P[t] += X[t] / nsamples

        reliability += np.sum(X < Pcrit) / (nsamples * myears)
        inertia += np.sum(np.absolute(np.diff(decisions)
                                      < 0.02)) / (nsamples * myears)
        utility += np.sum(alpha * decisions * np.power(delta,
                                                       np.arange(myears))) / nsamples
    max_P = np.max(average_daily_P)
    return max_P, utility, inertia, reliability

## 1. Connecting the lake model with the EMA workbench.

Given the Python implementation of the lake problem above, adapt the code and connect it to the EMA workbench
using the following lever ranges and uncertainty ranges:

|Levers 	|Range	        |Default value|
|-----------|--------------:|------------:|
|$r1$    	|0.0 – 2.0	    |0.5          |
|$r2$	    |0.0 – 2.0 	    |0.5.         |
|$c1$      	|-2 – 2	        |0.25         |
|$c2$	    |-2 – 2	        |0.25         |
|$w1$	    |0.0-1.0    	|0.5          |


|Uncertainties	|Range	        |Default value|
|---------------|--------------:|------------:|
|$\mu$    	    |0.01 – 0.05	|0.02         |
|$\sigma$	    |0.001 – 0.005 	|0.0017       |
|$b$      	    |0.1 – 0.45	    |0.42         |
|$q$	        |2 – 4.5	    |2            |
|$\delta$	    |0.93 – 0.99	|0.98         |
    
You can follow [this tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/directed-search.html)for guidance.

The outcomes in the EMA workbench refers to the objectives of the problem, in this case we have four. 1) maximum Phosphorous (to be minimized) 2) utility (to be maximized) 3) intertia (to be maximized), and reliability (to be maximized).  Use an alpha value of 0.41, with number of samples= 150, and number of years = 100.

## 2. How would you introduce a constrain within the optimization to reflect a desired performance threshold for a given objective?

You don't actually have to perform this step, simply specify how you would go about establishing a constraint in the optimization formulation, in such a way that it only finds solutions with maximum pollution (max phosphorous) of 0.85.

# 3.  Run the optimization and track the performance metrics

Tip: the EMA Workbench uses [Platypus](https://github.com/Project-Platypus/Platypus) to run the optimization via the evaluator class, you can also collect metrics during runtime specifying the convergence option. Below is a sample snippet on how to run the optimization and collect performance metrics during runtime. 

In [16]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

convergence_metrics = [HyperVolume(minimum=[0,0,0,0], maximum=[1,1.01,1.01,1.01]),
                       EpsilonProgress()]


ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(
        nfe=10000, 
        searchover='levers',
        epsilons=[0.1, 0.1, 0.01, 0.1],
        convergence=convergence_metrics,
        constraints=constraints)


# 4.  Selecting the objectives.

The outputs from the optimization runs will contain the decision variables (i.e. the parameters of the radial basis functions) and objectives combined.  Create a data structure that only contains the objective values without the decision variables. 

Tip: each row in the output matrix represents a different solution with it's obective values, the first columns are the decision variables and the last columns are the objective values. 

# 5.  Visualizing the results.

Present visually the results of the Pareto optimal solutions (in the objective space), feel free to be creative! Provide a brief discussion the results, are there any tradeoffs observed?

Tip: If you need inspiration check out the EMA workbench [parcoords](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/parcoords.html).

## 6. Establishing a performance threshold. 

Show visually only the solutions from the Pareto set that yield a reliability above 80%, and briefly discuss the results. 

# 7. Performance metrics

Show in a dataframe the results from the metrics (convergence) collected during the optimization. Plot the metrics (e.g hypervolume or epsilon progress) as a function of the number of function evaluations (nfe). Provide a brief discussion of the results.

### Extra credit: Visualize the phosphorous release from the policy with the highest reliability.   If you plot phosphorous release as a function of time, what do you observe? 

I want aknowledge my colleagues Jan Kwakkel and Giacomo Marangoni for their valuable contributions to this assignment. 